# import

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import interp1d

# Step 1: Read and filter the original files

In [14]:
# File path to the folder containing the original CSV files
dfl_file_path = 'N:/MSc_grad_proj/Data/dfl_traffic_count'
years_to_filter = [2018, 2019, 2021, 2022, 2023, 2024]

# List all files in the directory
file_paths = [os.path.join(dfl_file_path, file) for file in os.listdir(dfl_file_path) if file.endswith('.csv')]

# Initialize list to store filtered file paths
file_paths_filtered = []

for file_path in file_paths:
    # 读取CSV文件
    df = pd.read_csv(file_path, encoding='ISO-8859-1')  # 调整编码如果需要
    
    # 将'count_date'列转换为datetime类型
    df['count_date'] = pd.to_datetime(df['count_date'], errors='coerce')
    
    # 删除'all_motor_vehicles'等于0的行
    df = df[df['all_motor_vehicles'] != 0]
    
    # 过滤指定年份的行
    filtered_df = df[df['count_date'].dt.year.isin(years_to_filter)]
    
    # 保存过滤后的数据到新文件
    new_file_path = file_path.replace('.csv', '_filtered.csv')
    filtered_df.to_csv(new_file_path, index=False)
    print(f"Filtered data saved to {new_file_path}")
    
    # Append new file path to the filtered file paths list
    file_paths_filtered.append(new_file_path)


Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_103_filtered.csv
Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_107_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2204947645.py:13: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='ISO-8859-1')  # 调整编码如果需要


Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_109_filtered.csv
Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_110_filtered.csv
Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_145_filtered.csv
Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_174_filtered.csv
Filtered data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\dft_rawcount_local_authority_id_96_filtered.csv


In [3]:
file_paths

['N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_103.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_107.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_109.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_110.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_145.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_174.csv',
 'N:\\MSc_grad_proj\\Data\\dfl_traffic_count\\dft_rawcount_local_authority_id_96.csv']

In [15]:
file_paths_filtered

['N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_103_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_107_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_109_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_110_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_145_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_174_filtered.csv',
 'N:/MSc_grad_proj/Data/dfl_traffic_count\\dft_rawcount_local_authority_id_96_filtered.csv']

# Step 2: Processing filtered files

## Linear interpolation

In [19]:
# Function to perform linear interpolation
def linear_interpolation(df, date_col, value_cols):
    df[date_col] = pd.to_datetime(df[date_col])
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    # Create a complete date range
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # Create a new DataFrame with the complete date range
    full_df = pd.DataFrame(date_range, columns=[date_col])
    
    for col in value_cols:
        # Perform linear interpolation
        df.set_index(date_col, inplace=True)
        df = df.resample('D').mean()  # Resample to daily frequency and calculate mean for existing data
        df[col] = df[col].interpolate(method='linear')
        
        # Ensure interpolated values do not fall below the minimum original values
        min_value = df[col].min()
        df[col] = np.where(df[col] < min_value, min_value, df[col])
        
        # Round values to 2 decimal places
        df[col] = df[col].round(2)
        
        full_df = full_df.merge(df[col], left_on=date_col, right_on=date_col, how='left')
    
    return full_df

# Specify the columns to interpolate
value_columns = ['pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
                 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                 'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle',
                 'all_hgvs']

for file_path in file_paths_filtered:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Interpolate separately for each road type
    for road_type in df['road_type'].unique():
        road_df = df[df['road_type'] == road_type]
        interpolated_df = linear_interpolation(road_df, 'count_date', value_columns)
        
        # Calculate 'all_motor_vehicles' as the sum of specific columns
        interpolated_df['all_motor_vehicles'] = interpolated_df[value_columns].sum(axis=1)
        
        # Merge interpolated data back into the main DataFrame
        interpolated_df['road_type'] = road_type
        if road_type == df['road_type'].unique()[0]:
            full_interpolated_df = interpolated_df
        else:
            full_interpolated_df = pd.concat([full_interpolated_df, interpolated_df])
    
    # Construct new file path
    processed_file_path = file_path.replace('dft_rawcount_local_authority_id_', '')
    processed_file_path = processed_file_path.replace('145', 'Camden').replace('174', 'City of London').replace('96', 'Islington').replace('110', 'Kensington and Chelsea').replace('107', 'Lambeth').replace('103', 'Southwark').replace('109', 'Westminster')
    
    # Save the interpolated data back to a new CSV file
    full_interpolated_df.to_csv(processed_file_path, index=False)
    print(f"Processed data saved to {processed_file_path}")


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2802831072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2802831072.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('D').mean()  # Resample to daily frequency and calculate mean for existing data


KeyError: "None of ['count_date'] are in the columns"

## Polynomial Interpolation（多项式插值）

Polynomial Interpolation for Each Road Type Separately

In [23]:
# Function to perform polynomial interpolation
def polynomial_interpolation(df, date_col, value_cols, max_degree=3):
    df[date_col] = pd.to_datetime(df[date_col])
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    # Create a complete date range
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # Create a new DataFrame with the complete date range
    full_df = pd.DataFrame(date_range, columns=[date_col])
    
    for col in value_cols:
        # Fit a polynomial to the non-null data
        non_null_data = df.dropna(subset=[col])
        x = (non_null_data[date_col] - min_date).dt.days.values
        y = non_null_data[col].values
        
        # Adjust polynomial degree based on the number of data points
        degree = min(max_degree, len(x) - 1)
        if degree < 1:
            full_df[col] = np.nan
            continue
        
        polynomial = np.poly1d(np.polyfit(x, y, degree))
        
        # Apply the polynomial to the full date range
        full_df[col] = polynomial((full_df[date_col] - min_date).dt.days.values)
        
        # Ensure interpolated values do not fall below the minimum original values
        min_value = non_null_data[col].min()
        full_df[col] = np.where(full_df[col] < min_value, min_value, full_df[col])
        
        # Round values to 2 decimal places
        full_df[col] = full_df[col].round(2)
    
    return full_df

# Specify the columns to interpolate
value_columns = ['pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
                 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                 'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle',
                 'all_hgvs']

for file_path in file_paths_filtered:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Interpolate separately for each road type
    for road_type in df['road_type'].unique():
        road_df = df[df['road_type'] == road_type]
        interpolated_df = polynomial_interpolation(road_df, 'count_date', value_columns, max_degree=3)
        
        # Calculate 'all_motor_vehicles' as the sum of specific columns
        interpolated_df['all_motor_vehicles'] = interpolated_df[value_columns].sum(axis=1)
        
        # Merge interpolated data back into the main DataFrame
        interpolated_df['road_type'] = road_type
        if road_type == df['road_type'].unique()[0]:
            full_interpolated_df = interpolated_df
        else:
            full_interpolated_df = pd.concat([full_interpolated_df, interpolated_df])
    
    # Construct new file path
    processed_file_path = file_path.replace('dft_rawcount_local_authority_id_', '')
    processed_file_path = processed_file_path.replace('145', 'Camden').replace('174', 'City of London').replace('96', 'Islington').replace('110', 'Kensington and Chelsea').replace('107', 'Lambeth').replace('103', 'Southwark').replace('109', 'Westminster')
    
    # Save the interpolated data back to a new CSV file
    full_interpolated_df.to_csv(processed_file_path, index=False)
    print(f"Processed data saved to {processed_file_path}")


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])


Processed data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\Southwark_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])


Processed data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\Lambeth_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])


...

Processed data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\Kensington and Chelsea_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])


Processed data saved to N:/MSc_grad_proj/Data/dfl_traffic_count\Camden_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\2026349070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
c:\Program Files\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:667: RuntimeWarning: invalid value encountered in divide
  lhs /= scale


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
# Function to perform polynomial interpolation
def polynomial_interpolation(df, date_col, value_cols, degree=3):
    df[date_col] = pd.to_datetime(df[date_col])
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    # Create a complete date range
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # Create a new DataFrame with the complete date range
    full_df = pd.DataFrame(date_range, columns=[date_col])
    
    for col in value_cols:
        # Fit a polynomial to the non-null data
        non_null_data = df.dropna(subset=[col])
        x = (non_null_data[date_col] - min_date).dt.days.values
        y = non_null_data[col].values
        polynomial = np.poly1d(np.polyfit(x, y, degree))
        
        # Apply the polynomial to the full date range
        full_df[col] = polynomial((full_df[date_col] - min_date).dt.days.values)
        
        # Ensure interpolated values do not fall below the minimum original values
        min_value = non_null_data[col].min()
        full_df[col] = np.where(full_df[col] < min_value, min_value, full_df[col])
        
        # Round values to 2 decimal places
        full_df[col] = full_df[col].round(2)
    
    return full_df

# Specify the columns to interpolate
value_columns = ['pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
                 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                 'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle',
                 'all_hgvs']

for file_path in file_paths_filtered:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Interpolate separately for each road type
    for road_type in df['road_type'].unique():
        road_df = df[df['road_type'] == road_type]
        interpolated_df = polynomial_interpolation(road_df, 'count_date', value_columns, degree=3)
        
        # Calculate 'all_motor_vehicles' as the sum of specific columns
        interpolated_df['all_motor_vehicles'] = interpolated_df[value_columns].sum(axis=1)
        
        # Merge interpolated data back into the main DataFrame
        interpolated_df['road_type'] = road_type
        if road_type == df['road_type'].unique()[0]:
            full_interpolated_df = interpolated_df
        else:
            full_interpolated_df = pd.concat([full_interpolated_df, interpolated_df])
    
    # Construct new file path
    processed_file_path = file_path.replace('.csv', '_processed.csv')
    processed_file_path = processed_file_path.replace('dft_rawcount_local_authority_id_', '')
    processed_file_path = processed_file_path.replace('145', 'Camden').replace('174', 'City of London').replace('96', 'Islington').replace('110', 'Kensington and Chelsea').replace('107', 'Lambeth').replace('103', 'Southwark').replace('109', 'Westminster')
    
    # Save the interpolated data back to a new CSV file
    full_interpolated_df.to_csv(processed_file_path, index=False)
    print(f"Processed data saved to {processed_file_path}")


## Spline interpolation

In [21]:
# Function to perform spline interpolation
def spline_interpolation(df, date_col, value_cols):
    df[date_col] = pd.to_datetime(df[date_col])
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    # Create a complete date range
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # Create a new DataFrame with the complete date range
    full_df = pd.DataFrame(date_range, columns=[date_col])
    
    for col in value_cols:
        # Perform spline interpolation
        df.set_index(date_col, inplace=True)
        df = df.resample('D').mean()  # Resample to daily frequency and calculate mean for existing data
        df[col] = df[col].interpolate(method='spline', order=3)
        
        # Ensure interpolated values do not fall below the minimum original values
        min_value = df[col].min()
        df[col] = np.where(df[col] < min_value, min_value, df[col])
        
        # Round values to 2 decimal places
        df[col] = df[col].round(2)
        
        full_df = full_df.merge(df[col], left_on=date_col, right_on=date_col, how='left')
    
    return full_df

# Specify the columns to interpolate
value_columns = ['pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches',
                 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                 'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle',
                 'all_hgvs']

for file_path in file_paths_filtered:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Interpolate separately for each road type
    for road_type in df['road_type'].unique():
        road_df = df[df['road_type'] == road_type]
        interpolated_df = spline_interpolation(road_df, 'count_date', value_columns)
        
        # Calculate 'all_motor_vehicles' as the sum of specific columns
        interpolated_df['all_motor_vehicles'] = interpolated_df[value_columns].sum(axis=1)
        
        # Merge interpolated data back into the main DataFrame
        interpolated_df['road_type'] = road_type
        if road_type == df['road_type'].unique()[0]:
            full_interpolated_df = interpolated_df
        else:
            full_interpolated_df = pd.concat([full_interpolated_df, interpolated_df])
    
    # Construct new file path
    processed_file_path = file_path.replace('.csv', '_processed.csv')
    processed_file_path = processed_file_path.replace('dft_rawcount_local_authority_id_', '')
    processed_file_path = processed_file_path.replace('145', 'Camden').replace('174', 'City of London').replace('96', 'Islington').replace('110', 'Kensington and Chelsea').replace('107', 'Lambeth').replace('103', 'Southwark').replace('109', 'Westminster')
    
    # Save the interpolated data back to a new CSV file
    full_interpolated_df.to_csv(processed_file_path, index=False)
    print(f"Processed data saved to {processed_file_path}")


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\3818128250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_col] = pd.to_datetime(df[date_col])
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_1708\3818128250.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('D').mean()  # Resample to daily frequency and calculate mean for existing data


KeyError: "None of ['count_date'] are in the columns"

In [ ]:
'''
# Step 2: Process the filtered files
filtered_file_paths = [file.replace('.csv', '_filtered.csv') for file in file_paths]

for filtered_file_path in filtered_file_paths:
    # Check if filtered file exists
    if not os.path.exists(filtered_file_path):
        print(f"Filtered file {filtered_file_path} does not exist. Skipping.")
        continue
    
    # Read the filtered CSV file
    df = pd.read_csv(filtered_file_path, encoding='ISO-8859-1')  # Adjust encoding if necessary
    
    # Convert 'count_date' to datetime and extract the date part
    df['count_date'] = pd.to_datetime(df['count_date'], errors='coerce')
    df['date'] = df['count_date'].dt.date
    
    # Group by day and calculate the mean for specific columns
    columns_to_interpolate = [
        'pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 
        'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'hgvs_3_rigid_axle', 
        'hgvs_4_or_more_rigid_axle', 'hgvs_3_or_4_articulated_axle', 
        'hgvs_5_articulated_axle', 'hgvs_6_articulated_axle', 'all_hgvs', 
        'all_motor_vehicles'
    ]
    daily_df = df.groupby('date')[columns_to_interpolate].mean().reset_index()
    
    # Create a complete date range
    full_date_range = pd.date_range(start=daily_df['date'].min(), end=daily_df['date'].max())
    
    # Reindex to have a continuous date range
    daily_df = daily_df.set_index('date').reindex(full_date_range).reset_index()
    daily_df.rename(columns={'index': 'date'}, inplace=True)
    
    # Interpolate missing values using spline interpolation for specified columns
    for column in columns_to_interpolate:
        mask = np.isfinite(daily_df[column])
        x = np.arange(len(daily_df))
        spline = UnivariateSpline(x[mask], daily_df.loc[mask, column], s=0)
        daily_df[column] = spline(x)
        daily_df[column] = daily_df[column].round(2)  # Round to 2 decimal places
    
    # Save the processed data to a new CSV file
    processed_file_path = filtered_file_path.replace('_filtered.csv', '_processed.csv')
    daily_df.to_csv(processed_file_path, index=False)
    
    print(f"Processed data saved to {processed_file_path}")
'''